In [1]:
import scanpy as sc
from scFates.tools.utils import get_X
import pandas as pd
import numpy as np

In [2]:
adata=sc.read("data/adata_aligned.h5ad")

In [3]:
adata.obs["annot"]=adata.obs.leiden.astype(str).replace(["18","19"],["HC","OM"])

In [4]:
adata.obs.loc[~adata.obs.annot.isin(["HC","OM"]),"annot"]="SGN"

###### 2274-135

In [5]:
adata.obs.devtime.value_counts()

E18.5    677
E16.5    661
P3       500
E15.5    229
E14.5    135
E17.5     72
Name: devtime, dtype: int64

In [6]:
gr=adata.obs.groupby(["devtime","annot"])

In [7]:
gr.count()

batch  plate  n_genes_by_counts  total_counts  \
devtime annot                                                  
E14.5   HC         0      0                  0             0   
        OM        53     53                 53            53   
        SGN       82     82                 82            82   
E15.5   HC         0      0                  0             0   
        OM        68     68                 68            68   
        SGN      161    161                161           161   
E16.5   HC         8      8                  8             8   
        OM        73     73                 73            73   
        SGN      580    580                580           580   
E17.5   HC         1      1                  1             1   
        OM         0      0                  0             0   
        SGN       71     71                 71            71   
E18.5   HC        66     66                 66            66   
        OM         0      0                  0             0   
        SGN      611    611                611           611   
P3      HC         0      0                  0             0   
        OM         0      0                  0             0   
        SGN      500    500                500           500   

               total_counts_ERCC  pct_counts_ERCC  leiden  
devtime annot                                              
E14.5   HC                     0                0       0  
        OM                    53               53      53  
        SGN                   82               82      82  
E15.5   HC                     0                0       0  
        OM                    68               68      68  
        SGN                  161              161     161  
E16.5   HC                     8                8       8  
        OM                    73               73      73  
        SGN                  580              580     580  
E17.5   HC                     1                1       1  
        OM                     0                0       0  
        SGN                   71               71      71  
E18.5   HC                    66               66      66  
        OM                     0                0       0  
        SGN                  611              611     611  
P3      HC                     0                0       0  
        OM                     0                0       0  
        SGN                  500              500     500

In [8]:
cl=[]
for c in adata.obs.leiden.cat.categories:
    means=[]
    for l in get_X(adata,
                   adata.obs_names[adata.obs.leiden==c],
                   adata.var_names,
                   layer=None,
                   togenelist=True):
        means.append(np.mean(l))
    cl.append(pd.Series(means,index=adata.var_names,name=c))

clu_means=pd.concat(cl,axis=1)

In [9]:
clu_means.to_csv("data/allmeans.csv")

In [10]:
sc.tl.rank_genes_groups(adata,"leiden",method="wilcoxon")

/home/lfaure/miniconda3/envs/bio-gpu-18/lib/python3.8/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'annot' as categorical


In [11]:
writer = pd.ExcelWriter('data/All_DE.xlsx', engine='xlsxwriter')
for cl in adata.obs.leiden.cat.categories:
    sc.get.rank_genes_groups_df(adata,cl,pval_cutoff=0.05,log2fc_min=0).set_index("names").to_excel(writer, 
                                                                     sheet_name="Leiden %s" %cl)
    
writer.save()

In [12]:
adata=adata[~adata.obs.leiden.isin(["18","19"])]
sc.tl.rank_genes_groups(adata,"leiden",method="wilcoxon")

Trying to set attribute `.uns` of view, copying.


In [13]:
writer = pd.ExcelWriter('data/SGN_DE.xlsx', engine='xlsxwriter')
for cl in adata.obs.leiden.cat.categories:
    sc.get.rank_genes_groups_df(adata,cl,pval_cutoff=0.05,log2fc_min=0).set_index("names").to_excel(writer, 
                                                                     sheet_name="Leiden %s" %cl)
    
writer.save()

In [14]:
adata=sc.read("data/adata_tree_fitted.h5ad")
adata.var[["A","fdr"]].to_csv("data/all_diff_pseudo.csv")